In [9]:
import numpy as np
from qiskit_aer import Aer
from qiskit.primitives import StatevectorEstimator

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp


qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)

obs = SparsePauliOp("ZZ")

num_shots = 1000

# Statevector Estimator
# Dict for Options unfortunatly not available ...
estimator = StatevectorEstimator(seed=1, default_precision=1 / np.sqrt(num_shots))
job = estimator.run([(qc, obs)])
result = job.result()
result

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>)), metadata={'target_precision': 0.03162277660168379, 'circuit_metadata': {}})], metadata={'version': 2})

In [10]:
from squlearn import Executor
executor = Executor(estimator)
executor.shots

Statevector Estimator detected, setting backend to Aer simulator


1000

In [3]:
# set seed after creation (probably not needed)
estimator._seed = 1234
num_shots = 321
job = estimator.run([(qc, obs)], precision=1 / np.sqrt(num_shots))
result = job.result()[0]
result.data.evs

array(-0.08951744)

In [4]:
from qiskit.primitives import BackendEstimatorV2  # Only available at Qiskit 1.2


backend = Aer.get_backend("qasm_simulator")
# this is how to set a seed for the qasm simulator, default shots can be set with default_shots
num_shots = 1234
sampler = BackendEstimatorV2(
    backend=backend, options={"seed_simulator": 1, "default_precision": 1 / np.sqrt(num_shots)}
)


job = sampler.run([(qc, obs)])  # , precision=1/np.sqrt(num_shots))
result = job.result()[0]
print("evs:", result.data.evs)
print("performed shots:", result.metadata["shots"])

evs: -0.03805668016194332
performed shots: 1235


In [5]:
from qiskit_ibm_runtime import EstimatorV2, EstimatorOptions
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_ibm_runtime.options.utils import Unset

# this is similar to the old optimization level of the V1 Primitives
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

fake_backend = FakeManilaV2()

# In case of the fake backend, the seed and the default precision is set in the options
# options = {"simulator": {"seed_simulator": 42}, "default_precision": 1 / np.sqrt(num_shots)}
#options = EstimatorOptions(simulator={"seed_simulator": 42}, default_precision=1 / np.sqrt(num_shots))
options = EstimatorOptions(simulator={"seed_simulator": 42}, default_shots=1234)

estimator = EstimatorV2(mode=fake_backend, options=options)

# transpiling has to be done before running the job :/
pm = generate_preset_pass_manager(backend=fake_backend, optimization_level=1)
isa_qc = pm.run(qc)
isa_observables = obs.apply_layout(isa_qc.layout)

job = estimator.run([(isa_qc, isa_observables)])
result = job.result()[0]
result.data.evs

array(-0.03481781)

In [6]:
from squlearn import Executor
executor = Executor(estimator, qpu_parallelization="auto")
estimator_exec = executor.get_estimator_v2()

job = estimator_exec.run([(qc, obs)])  # , precision=1/np.sqrt(num_shots))
result = job.result()[0]
print("evs:", result.data.evs)
result

AttributeError: 'EstimatorOptions' object has no attribute 'get'

In [ ]:
job = estimator.run([(isa_qc, isa_observables)], precision=1 / np.sqrt(100000))
result = job.result()[0]
result.data.evs

AttributeError: 'EstimatorOptions' object has no attribute 'get'

In [ ]:
from squlearn import Executor

backend = Aer.get_backend("qasm_simulator")
# this is how to set a seed for the qasm simulator, default shots can be set with default_shots
num_shots = 1234
estimator = BackendEstimatorV2(
    backend=backend, options={"seed_simulator": 1, "default_precision": 1 / np.sqrt(num_shots)}
)

executor = Executor(estimator, cache_dir="cache", caching=True, log_file="log.txt")


estimator_exec = executor.get_estimator_v2()

job = estimator_exec.run([(qc, obs)])  # , precision=1/np.sqrt(num_shots))
result = job.result()[0]
print("evs:", result.data.evs)
result

evs: -0.03805668016194332


PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=0.028434905975046908), metadata={'target_precision': 0.028467047115478598, 'shots': 1235, 'circuit_metadata': {}})